In [ ]:
from sklearn.linear_model import RidgeCV
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
pd.options.mode.chained_assignment = None
# from sklearn.pipeline import make_pipeline
# model = make_pipeline(StandardScaler(with_mean=False), _RidgeGCV())

teams_dict, teams_list = get_teams(league="NBA")
box_DIR = "../team_ratings/boxscores/"

In [ ]:
def time_decay(X):
    X = X.to_numpy()
    y = np.zeros(np.size(X))
    for i,x in enumerate(X):
        yj=0
        for j in range(i):
            xj = x-X[j]
            yj += -np.exp(-1*(x-X[j]))
        y[i] = yj
    return y

In [ ]:
season = "2022"

In [ ]:
df1 = pd.read_csv(box_DIR + f"NBA_BoxScores_Adv_{season}.csv")
df1 = df1.rename(
    columns={
        "offensiveRating": "ORtg",
        "defensiveRating": "DRtg",
        "netRating": "NRtg",
        "possessions": "poss",
    }
)
cols = ["gameId", "teamId", "ORtg", "DRtg", "NRtg", "poss"]
df1 = df1[cols]
df1["Win"] = df1["NRtg"] > 0
df1["Loss"] = df1["NRtg"] < 0
df2 = pd.read_csv(box_DIR + f"NBA_BoxScores_Standard_{season}.csv")
df2 = df2.rename(
    columns={
        "GAME_ID": "gameId",
        "TEAM_ID": "teamId",
        "TEAM_ABBREVIATION": "teamTricode",
        "GAME_DATE": "gameDate",
        "TEAM_NAME": "teamName",
        "MATCHUP": "Matchup",
    }
)
cols2 = ["gameId", "teamId", "teamTricode", "gameDate","Matchup"]
df2 = df2[cols2]
df2["Home"] = np.where(~df2["Matchup"].str.contains("@"), 1, 0)
df2["Away"] = np.where(df2["Matchup"].str.contains("@"), 1, 0)
df2["gameId"] = df2["gameId"].astype(int)
df3 = pd.merge(df2, df1, on=["gameId", "teamId"])
df3 = df3.rename(columns={"gameDate": "Date"})
df3["Date"] = pd.to_datetime(df3["Date"], format="%Y-%m-%d")
df4 = df3.sort_values(by=["teamTricode","Date"])
df4["Date_D"] = (df4["Date"] - df4["Date"].iloc[0]).dt.days
teams = df4["teamTricode"].unique()
dfa = []
for team in teams:
    df5 = df4[df4["teamTricode"] == team].reset_index(drop=True)
    df5["Rest"] = time_decay(df5['Date_D'])
    df5["Rest"] = df5["Rest"].round(5)
    dfa.append(df5)
df5 = pd.concat(dfa)

In [ ]:
df5 = df5.reset_index(drop=True)
cols = [
    "gameId",
    "teamTricode",
    "teamId",
    "ORtg",
    "DRtg",
    "NRtg",
    "poss",
    "Home",
    "Away",
    "Rest",
    "Date",
]
df6 = df5[cols]
df6[cols[1:]] = df6[cols[1:]].astype(str)
df7 = df6.groupby("gameId")[cols[1:]].agg(", ".join).reset_index()
df8 = df7.copy()
df8[["team1", "team2"]] = df7["teamTricode"].str.split(",", expand=True)
df8[["tId1", "tId2"]] =   df7["teamId"].str.split(",", expand=True)
df8[["ORtg1", "ORtg2"]] = df7["ORtg"].str.split(",", expand=True)
df8[["DRtg1", "DRtg2"]] = df7["DRtg"].str.split(",", expand=True)
df8[["NRtg1", "NRtg2"]] = df7["NRtg"].str.split(",", expand=True)
df8[["poss1", "poss2"]] = df7["poss"].str.split(",", expand=True)
df8[["rest1", "rest2"]] = df7["Rest"].str.split(",", expand=True)
df8[["date1", "date2"]] = df7["Date"].str.split(",", expand=True)
df8[["home1", "home2"]] = df7["Home"].str.split(",", expand=True)
# df8[["away1", "away2"]] = df7["Away"].str.split(",", expand=True)
df8 = df8.drop(columns=cols[1:])
df9 = df7.copy()
df9[["team2", "team1"]] = df7["teamTricode"].str.split(",", expand=True)
df9[["tId2", "tId1"]] =   df7["teamId"].str.split(",", expand=True)
df9[["ORtg2", "ORtg1"]] = df7["ORtg"].str.split(",", expand=True)
df9[["DRtg2", "DRtg1"]] = df7["DRtg"].str.split(",", expand=True)
df9[["NRtg2", "NRtg1"]] = df7["NRtg"].str.split(",", expand=True)
df9[["poss2", "poss1"]] = df7["poss"].str.split(",", expand=True)
df9[["rest2", "rest1"]] = df7["Rest"].str.split(",", expand=True)
df9[["date1", "date2"]] = df7["Date"].str.split(",", expand=True)
df9[["home2", "home1"]] = df7["Home"].str.split(",", expand=True)
# df9[["away2", "away1"]] = df7["Away"].str.split(",", expand=True)
df9 = df9.drop(columns=cols[1:])
df10 = pd.concat([df8, df9]).sort_values(by="gameId").reset_index(drop=True)
cols = df10.columns
df10[cols[3:5]]   = df10[cols[3:5]].astype(int)
df10[cols[5:15]]  = df10[cols[5:15]].astype(float)


In [ ]:
df10["date1"] = pd.to_datetime(df10["date1"], format="%Y-%m-%d")
df10[cols[17:19]] = df10[cols[17:19]].astype(int)
df10[cols[17:19]] = df10[cols[17:19]].astype(bool)
df10["rest_diff"] = df10["rest2"] - df10["rest1"]
df10 = df10.drop(columns=["date2"])
df10 = df10.rename(columns={"date1":"Date"})
data = df10.copy()

In [ ]:
df1 = data.copy()
df1["pts1"] = df1["ORtg1"] * df1["poss1"] 
df1["pts2"] = df1["DRtg1"] * df1["poss1"]
off_prior = df1.groupby(["tId1"])[["poss1", "pts1"]].agg("sum").reset_index()
def_prior = df1.groupby(["tId1"])[["poss1", "pts2"]].agg("sum").reset_index()
off_prior["OFF"] = off_prior["pts1"] / off_prior["poss1"]
off_prior["OFF"] = off_prior["OFF"].round(3)
off_prior = off_prior[["tId1", "OFF"]]
def_prior["DEF"] = def_prior["pts2"] / def_prior["poss1"]
def_prior["DEF"] = def_prior["DEF"].round(3)
def_prior = def_prior[["tId1", "DEF"]]
# def_prior = def_prior.rename(columns={"tId1":"tId2"})
df1 = pd.merge(df1,off_prior, on="tId1")
df1 = pd.merge(df1,def_prior, on="tId1")
df1["NET"] = df1["NRtg1"] - (df1["OFF"] - df1["DEF"])

In [ ]:
df1 = data.query("home1").reset_index(drop=True)
df1["pts1"] = df1["ORtg1"] * df1["poss1"] 
df1["pts2"] = df1["DRtg1"] * df1["poss1"]
off_prior = df1.groupby(["tId1"])[["poss1", "pts1"]].agg("sum").reset_index()
def_prior = df1.groupby(["tId1"])[["poss1", "pts2"]].agg("sum").reset_index()
off_prior["OFF"] = off_prior["pts1"] / off_prior["poss1"]
off_prior["OFF"] = off_prior["OFF"].round(3)
off_prior = off_prior[["tId1", "OFF"]]
def_prior["DEF"] = def_prior["pts2"] / def_prior["poss1"]
def_prior["DEF"] = def_prior["DEF"].round(3)
def_prior = def_prior[["tId1", "DEF"]]
def_prior = def_prior.rename(columns={"tId1":"tId2"})
df1 = pd.merge(df1,off_prior, on="tId1")
df1 = pd.merge(df1,def_prior, on="tId2")
df2 = data.query("home2").reset_index(drop=True)
df2["pts1"] = df2["ORtg1"] * df2["poss1"] 
df2["pts2"] = df2["DRtg1"] * df2["poss1"]
off_prior = df2.groupby(["tId1"])[["poss1", "pts1"]].agg("sum").reset_index()
def_prior = df2.groupby(["tId1"])[["poss1", "pts2"]].agg("sum").reset_index()
off_prior["OFF"] = off_prior["pts1"] / off_prior["poss1"]
off_prior["OFF"] = off_prior["OFF"].round(3)
off_prior = off_prior[["tId1", "OFF"]]
def_prior["DEF"] = def_prior["pts2"] / def_prior["poss1"]
def_prior["DEF"] = def_prior["DEF"].round(3)
def_prior = def_prior[["tId1", "DEF"]]
# def_prior = def_prior.rename(columns={"tId1":"tId2"})
df2 = pd.merge(df2,off_prior, on="tId1")
df2 = pd.merge(df2,def_prior, on="tId1")
df1= pd.concat([df1,df2])
df1["NET"] = df1["NRtg1"] - (df1["OFF"] - df1["DEF"])
df1["rest_adv"] =  df1["rest_diff"]>0.2
df1["rest_dadv"] = df1["rest_diff"]<-0.2

In [ ]:
df2 = df1.groupby("tId1")[["rest_adv","rest_dadv"]].agg({"rest_adv":["sum"],"rest_dadv":["sum"]})
df2.columns = ["rest_adv","rest_dadv"]
df2 = df2.reset_index()

In [ ]:
df2["Team"] = df2["tId1"].map(teams_dict)

In [ ]:
df2.sort_values("rest_adv",ascending=False)

In [ ]:
i = 0
# data.query(f"team1 == '{teams[i]}'").sort_values(by=["tId1","Date"])

In [ ]:
df2 = df1.query("rest_diff>0")

In [ ]:
df2["NET"].mean()

In [ ]:
df1.columns

In [ ]:
i = 1
df3 = data.query(f"team1 == '{teams[i]}'").sort_values(by=["tId1","Date"])

In [ ]:
sns.scatterplot(data=df3, x="Date", y = "rest1")
sns.lineplot(data=df3, x="Date", y = "rest1")

In [ ]:
sns.scatterplot(data=df3, x="Date", y = "rest_diff")
sns.lineplot(data=df3, x="Date", y = "rest_diff")
sns.lineplot(data=df3, x="Date", y = 0, color = "black")
sns.lineplot(data=df3, x="Date", y = -0.1, linestyle = "--", color = "black")
sns.lineplot(data=df3, x="Date", y = +0.1, linestyle = "--", color = "black")